In [59]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hamming_loss, precision_recall_curve
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch
from datasets import Dataset
import nlpaug.augmenter.word as naw
from nlpaug.util import Action
import optuna

In [60]:
# ==================================================
# 0. Choose Model: "modernbert", "roberta", or "legalbert"
# ==================================================
model_choice = "legalbert"  # change to "legalbert" or "modernbert" as desired

if model_choice == "roberta":
    model_name = "roberta-base"
elif model_choice == "legalbert":
    model_name = "nlpaueb/legal-bert-base-uncased"
elif model_choice == "modernbert":
    model_name = "answerdotai/ModernBERT-base"
else:
    raise ValueError("Invalid model_choice. Choose 'modernbert', 'roberta', or 'legalbert'.")


In [61]:
# -----------------------------
# 1. Load Data
# -----------------------------
def load_license_data(json_folder):
    license_data = []
    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            license_name = filename[:-5]
            filepath = os.path.join(json_folder, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
                license_data.append({
                    "license_name": license_name,
                    "family": data["family"],
                    "labels": data["labels"],
                    "text": data["text"],
                })
    return license_data

json_folder = "../../data/processed/preprocessed_licenses_json_2"
license_data = load_license_data(json_folder)
df = pd.DataFrame(license_data)

# Drop rows with missing labels
df.dropna(subset=["labels"], inplace=True)
df = df[df["labels"].apply(lambda x: len(x) > 0)]

In [62]:
# -----------------------------
# 2. Encode Labels
# -----------------------------
mlb = MultiLabelBinarizer()
df["labels"] = list(mlb.fit_transform(df["labels"]))
num_labels = len(mlb.classes_)


In [63]:
# -----------------------------
# 3. Split Data
# -----------------------------
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [64]:
# -----------------------------
# 4. BERT-Based Augmentation
# -----------------------------

def augment_text(text, aug_p=0.2, aug_max=3):
    """Context-aware augmentation using BERT"""
    aug = naw.ContextualWordEmbsAug(
        model_path='bert-base-uncased',
        action=Action.SUBSTITUTE,
        aug_p=aug_p,
        aug_max=aug_max,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    return aug.augment(text)[0]  # Return first augmented version

def augment_minority_classes(df, mlb, min_samples=50, aug_factor=3):
    augmented_texts = []
    augmented_labels = []
    
    # Calculate label counts (fixed)
    label_matrix = np.array(df["labels"].tolist())
    label_counts = label_matrix.sum(axis=0)
    minority_labels = np.where(label_counts < min_samples)[0]
    
    for _, row in df.iterrows():
        text = row["text"]
        labels = row["labels"]
        
        # Check if sample contains minority labels
        if any(label in minority_labels for label in np.where(labels == 1)[0]):
            for _ in range(aug_factor):
                new_text = augment_text(text)
                augmented_texts.append(new_text)
                augmented_labels.append(labels)
    
    # Create augmented DataFrame
    augmented_df = pd.DataFrame({
        "text": augmented_texts,
        "labels": list(augmented_labels)
    })
    
    return pd.concat([df, augmented_df], ignore_index=True)

# Apply augmentation
print("Original training size:", len(train_df))
train_df = augment_minority_classes(
    train_df,
    mlb=mlb,
    min_samples=80,
    aug_factor=5
)
print("Augmented training size:", len(train_df))

Original training size: 247
Augmented training size: 972


In [68]:
# -----------------------------
# 5. Tokenization
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Prepare datasets
train_dataset = Dataset.from_pandas(train_df).map(tokenize, batched=True)
val_dataset = Dataset.from_pandas(val_df).map(tokenize, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize, batched=True)

# Set format (ensure "labels" column)
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

loading configuration file config.json from cache at C:\Users\NPARSHO\.cache\huggingface\hub\models--nlpaueb--legal-bert-base-uncased\snapshots\15b570cbf88259610b082a167dacc190124f60f6\config.json
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file config.json from cache at C:\Users\NPARSHO\.cache\huggingface\

Map:   0%|          | 0/972 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [69]:
# -----------------------------
# 6. Focal Loss Implementation
# -----------------------------
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, labels):
        bce_loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels, reduction="none"
        )
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()

In [70]:
# -----------------------------
# 7. Custom Trainer
# -----------------------------
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Apply focal loss
        loss_fct = FocalLoss(alpha=0.25, gamma=2.0)
        loss = loss_fct(logits, labels.float())
        
        return (loss, outputs) if return_outputs else loss

In [73]:

# -----------------------------
# 8. Model Setup
# -----------------------------
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        problem_type="multi_label_classification",
        label2id={label: str(i) for i, label in enumerate(mlb.classes_)},
        id2label={str(i): label for i, label in enumerate(mlb.classes_)}
    )

In [ ]:
# -----------------------------
# 9. Training
# -----------------------------
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions)).cpu().numpy() > 0.5
    labels = p.label_ids
    
    return {
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "hamming_loss": hamming_loss(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="../../model/Bert4.0",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    save_total_limit=2,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    seed=42,
    log_level="info",       # Show detailed logs
    disable_tqdm=False
)

trainer = CustomTrainer(
    model=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\transformers\training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices


In [29]:
# -----------------------------
# 10. Train the Model
# -----------------------------
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: license_name, family, text. If license_name, family, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 972
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 610
  Number of trainable parameters = 124,667,933


  0%|          | 0/610 [00:00<?, ?it/s]

{'loss': 0.0405, 'grad_norm': 0.11439289897680283, 'learning_rate': 1.9672131147540985e-05, 'epoch': 0.16}
{'loss': 0.0271, 'grad_norm': 0.0758814811706543, 'learning_rate': 1.934426229508197e-05, 'epoch': 0.33}
{'loss': 0.0236, 'grad_norm': 0.07778170704841614, 'learning_rate': 1.9016393442622952e-05, 'epoch': 0.49}
{'loss': 0.0204, 'grad_norm': 0.06722109019756317, 'learning_rate': 1.8688524590163936e-05, 'epoch': 0.66}
{'loss': 0.0188, 'grad_norm': 0.0932110995054245, 'learning_rate': 1.836065573770492e-05, 'epoch': 0.82}
{'loss': 0.018, 'grad_norm': 0.07130974531173706, 'learning_rate': 1.8032786885245903e-05, 'epoch': 0.98}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: family, license_name, __index_level_0__, text. If family, license_name, __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 53
  Batch size = 16


  0%|          | 0/4 [00:00<?, ?it/s]

...

Saving model checkpoint to ../../model/Bert4.0\checkpoint-610
Configuration saved in ../../model/Bert4.0\checkpoint-610\config.json


{'eval_loss': 0.014367332682013512, 'eval_f1_macro': 0.5897142482466144, 'eval_f1_micro': 0.8600770218228498, 'eval_hamming_loss': 0.07091737150292778, 'eval_runtime': 52.4383, 'eval_samples_per_second': 1.011, 'eval_steps_per_second': 0.076, 'epoch': 10.0}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ../../model/Bert4.0\checkpoint-549 (score: 0.6004805539312504).


{'train_runtime': 29038.2895, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.021, 'train_loss': 0.00911996128251318, 'epoch': 10.0}


TrainOutput(global_step=610, training_loss=0.00911996128251318, metrics={'train_runtime': 29038.2895, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.021, 'total_flos': 2558059437957120.0, 'train_loss': 0.00911996128251318, 'epoch': 10.0})

In [ ]:
trainer.save_model("../../model/Roberta")  # Saves model and tokenizer
tokenizer.save_pretrained("../../model/Roberta")  # Save tokenizer

Saving model checkpoint to ../../model/Roberta
Configuration saved in ../../model/Roberta\config.json


('../../model/Roberta\\tokenizer_config.json',
 '../../model/Roberta\\special_tokens_map.json',
 '../../model/Roberta\\vocab.json',
 '../../model/Roberta\\merges.txt',
 '../../model/Roberta\\added_tokens.json',
 '../../model/Roberta\\tokenizer.json')

In [31]:
# -----------------------------
# 11. Threshold Optimization
# -----------------------------
def optimize_thresholds(y_true, y_pred_proba):
    thresholds = {}
    for i in range(y_true.shape[1]):
        precision, recall, th = precision_recall_curve(y_true[:, i], y_pred_proba[:, i])
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
        optimal_th = th[np.argmax(f1_scores)]
        thresholds[i] = optimal_th
    return thresholds

# Get validation predictions
val_preds = trainer.predict(val_dataset)
val_pred_proba = torch.sigmoid(torch.tensor(val_preds.predictions)).numpy()
val_labels = val_preds.label_ids

# Find optimal thresholds
optimal_thresholds = optimize_thresholds(val_labels, val_pred_proba)

# -----------------------------
# 12. Final Evaluation
# -----------------------------
test_preds = trainer.predict(test_dataset)
test_pred_proba = torch.sigmoid(torch.tensor(test_preds.predictions)).numpy()
test_labels = test_preds.label_ids

# Apply optimized thresholds
final_test_preds = (test_pred_proba > np.array(list(optimal_thresholds.values()))).astype(int)

# Print final metrics
print("\n--- Final Test Metrics ---")
print(f"F1 Macro: {f1_score(test_labels, final_test_preds, average='macro'):.4f}")
print(f"F1 Micro: {f1_score(test_labels, final_test_preds, average='micro'):.4f}")
print(f"Hamming Loss: {hamming_loss(test_labels, final_test_preds):.4f}")

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: family, license_name, __index_level_0__, text. If family, license_name, __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 53
  Batch size = 16


  0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: family, license_name, __index_level_0__, text. If family, license_name, __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction ****

  0%|          | 0/4 [00:00<?, ?it/s]


--- Final Test Metrics ---
F1 Macro: 0.6237
F1 Micro: 0.6896
Hamming Loss: 0.2075


c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [78]:
model_directory = "../../model/LegalBert"
model = AutoModelForSequenceClassification.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained(model_directory)

loading configuration file ../../model/LegalBert\config.json
Model config BertConfig {
  "_name_or_path": "../../model/LegalBert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Anti-DRM (obligation/WARNING)",
    "1": "Anti-Tivoization (obligation/WARNING)",
    "2": "Copyleft (network protective) (obligation/ALARM)",
    "3": "Copyleft (strong) (obligation/WARNING)",
    "4": "Copyleft (weak) (obligation/WARNING)",
    "5": "Declare modification in source code (obligation/WARNING)",
    "6": "Deprecated License (other/INFORMATION)",
    "7": "Display acknowledgement message (obligation/WARNING)",
    "8": "Display additional information (obligation/WARNING)",
    "9": "Display copyright notice (obligation/INFORMATION)",
    "10": "Display license in binary (o

In [79]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [80]:
# Get validation predictions
val_preds = trainer.predict(val_dataset)
val_pred_proba = torch.sigmoid(torch.tensor(val_preds.predictions)).numpy()
val_labels = val_preds.label_ids

# Find optimal thresholds
optimal_thresholds = optimize_thresholds(val_labels, val_pred_proba)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: family, license_name, __index_level_0__, text. If family, license_name, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 53
  Batch size = 16


  0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [81]:


# -----------------------------
# 12. Final Evaluation
# -----------------------------
test_preds = trainer.predict(test_dataset)
test_pred_proba = torch.sigmoid(torch.tensor(test_preds.predictions)).numpy()
test_labels = test_preds.label_ids

# Apply optimized thresholds
final_test_preds = (test_pred_proba > np.array(list(optimal_thresholds.values()))).astype(int)

# Print final metrics
print("\n--- Final Test Metrics ---")
print(f"F1 Macro: {f1_score(test_labels, final_test_preds, average='macro'):.4f}")
print(f"F1 Micro: {f1_score(test_labels, final_test_preds, average='micro'):.4f}")
print(f"Hamming Loss: {hamming_loss(test_labels, final_test_preds):.4f}")

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: family, license_name, __index_level_0__, text. If family, license_name, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 54
  Batch size = 16


  0%|          | 0/4 [00:00<?, ?it/s]


--- Final Test Metrics ---
F1 Macro: 0.6397
F1 Micro: 0.6956
Hamming Loss: 0.2018


c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [75]:
# Define your hyperparameter search space
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 5, 15),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.1, log=True),
        # Focal loss parameters: 
        "alpha": trial.suggest_float("alpha", 0.25, 0.5),
        "gamma": trial.suggest_float("gamma", 1.5, 3.0)
    }

training_args = TrainingArguments(
    output_dir="../../model/hp_search",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
    seed=42,
    # These values can be overridden by hyperparameter search
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
)

# Initialize the Trainer with your custom Trainer (using FocalLoss) and model_init
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Run hyperparameter search with Optuna
best_trial = trainer.hyperparameter_search(
    direction="maximize",  # Assuming you're maximizing F1 score
    hp_space=hp_space,
    n_trials=20  # You can adjust the number of trials
)

print("Best hyperparameters found:", best_trial.hyperparameters)


PyTorch: setting up devices
loading configuration file config.json from cache at C:\Users\NPARSHO\.cache\huggingface\hub\models--nlpaueb--legal-bert-base-uncased\snapshots\15b570cbf88259610b082a167dacc190124f60f6\config.json
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Anti-DRM (obligation/WARNING)",
    "1": "Anti-Tivoization (obligation/WARNING)",
    "10": "Display license in binary (obligation/INFORMATION)",
    "11": "Display license in the source (obligation/INFORMATION)",
    "12": "Doing Business with US (other/ALARM)",
    "13": "Endorsement prohibited (prohibition/INFORMATION)",
    "14": "Jurisdiction specific (other/WARNING)",
    "15": "Keep copy of source code available (obliga

  0%|          | 0/732 [00:00<?, ?it/s]

Attempting to create safetensors variant
Safetensors PR exists
Attempting to create safetensors variant
Safetensors PR exists
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: family, license_name, __index_level_0__, text. If family, license_name, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 53
  Batch size = 16


  0%|          | 0/4 [00:00<?, ?it/s]

Saving model checkpoint to ../../model/hp_search\run-0\checkpoint-61
Configuration saved in ../../model/hp_search\run-0\checkpoint-61\config.json


...

{'train_runtime': 25961.6358, 'train_samples_per_second': 0.449, 'train_steps_per_second': 0.028, 'train_loss': 0.006545368439512826, 'epoch': 12.0}


loading configuration file config.json from cache at C:\Users\NPARSHO\.cache\huggingface\hub\models--nlpaueb--legal-bert-base-uncased\snapshots\15b570cbf88259610b082a167dacc190124f60f6\config.json
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Anti-DRM (obligation/WARNING)",
    "1": "Anti-Tivoization (obligation/WARNING)",
    "10": "Display license in binary (obligation/INFORMATION)",
    "11": "Display license in the source (obligation/INFORMATION)",
    "12": "Doing Business with US (other/ALARM)",
    "13": "Endorsement prohibited (prohibition/INFORMATION)",
    "14": "Jurisdiction specific (other/WARNING)",
    "15": "Keep copy of source code available (obligation/WARNING)",
    "16": "L

  0%|          | 0/1586 [00:00<?, ?it/s]

Attempting to create safetensors variant
Safetensors PR exists
[W 2025-03-14 00:53:52,046] Trial 4 failed with parameters: {'learning_rate': 1.8983211874813515e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 13, 'weight_decay': 0.016436174343137766, 'alpha': 0.4312776160305887, 'gamma': 1.6192230403063839} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\transformers\integrations\integration_utils.py", line 249, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "c:\Users\NPARSHO\AppData\Local\anaconda3\envs\cuda2\Lib\site-packages\transformers\trainer.py", line 2163, in train
    return inner_training_loop(
           ^^^^^^^^^^^

KeyboardInterrupt: 

In [77]:
tokenizer.save_pretrained("../../model/LegalBert")

('../../model/LegalBert\\tokenizer_config.json',
 '../../model/LegalBert\\special_tokens_map.json',
 '../../model/LegalBert\\vocab.txt',
 '../../model/LegalBert\\added_tokens.json',
 '../../model/LegalBert\\tokenizer.json')